# Visualizing a Shapefile

This will be a short tutorial showing you how to code a program to view your own shapefile

In [3]:
import os, sys
import ipywidgets as ipyw
from osgeo import ogr, osr
from ipyleaflet import Map, GeoJSON
import zipfile
import shapely.wkt
import geojson

In [16]:
# file path of the shapefile

# polygon geometry shapefile
zip_file = os.path.abspath('./ne_10m_admin_0_boundary_lines_land.zip')
shape_name = "ne_110m_admin_0_countries"

# line geometry shapefile
# zip_file = os.path.abspath('./ne_10m_admin_0_boundary_lines_land.zip')
# shape_name = "ne_10m_admin_0_boundary_lines_land"

folder_path = os.path.dirname(zip_file)

# unzips shapefile and extracts the files to the current directory.
with zipfile.ZipFile(zip_file, 'r') as f:
    f.extractall(folder_path)

folder_path += "/" + shape_name # path containing unzipped files

This is where we actually do the work to render the image. To accomplish that, we use the library osgeo. ogr and osr are derived from osgeo. ogr and osr deal with the vector data inside of the shape file. 

In [17]:
# opens the file and extracts a layer from it
driver = ogr.GetDriverByName('ESRI Shapefile')
data_source = driver.Open(folder_path + '/' + shape_name + '.shp', 0)
layer = data_source.GetLayer()

# tells us the type of objects contained in the shapefile. 1 is points, 2 is lines, 3 is polygons
shape_type = layer.GetGeomType()
display(shape_type)

3

The next important thing to deal with are possible different spatial references. Depending on where the data was gathered or from where it was extracted, the spatial reference of the shapefile might not be optimal for our purposes. It is important to know which spatial reference was used because if the spatial reference system used in the shapefile is different from the spatial reference we are using for the map, we most likely will end up with a different result than we were expecting. Many things could go wrong such as the coordinates are completely off or that the data is completely misaligned. By transforming the spatial reference to the one we want, we avoid that problem.

In [18]:
# gets all the spatial references to analyze the data and applies them to a transformed coordinate system
# inner_spatial_reference is the source projection and outer_spatial_reference is the target projection
# in our case, we are changing the spatial reference to the WGS84 projection
inner_spatial_reference = layer.GetSpatialRef()
outer_spatial_reference = osr.SpatialReference()
outer_spatial_reference.ImportFromWkt(
    'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'
)
transform = osr.CoordinateTransformation(inner_spatial_reference, outer_spatial_reference)

# these are just placeholders to make information dumping easier. they will be used to store the individual
# feature properties and information in the shapefile
shapes = []
feature_collection = None

# these are here to make the visualization better. all they are used for is to create our own bounds so that all
# the information is visible when the map is rendered
x_min = sys.float_info.max
x_max = -sys.float_info.max
y_min = sys.float_info.max
y_max = -sys.float_info.max

# iterates through the opened layer to get all the feature headings. it's kind of like the keys
# in a dictionary or the fieldnames of a csv file. we're storing those into field_list
field_list = []
for i in range(layer.GetLayerDefn().GetFieldCount()):
    field_list.append(layer.GetLayerDefn().GetFieldDefn(i).GetName())

Think of this next part as kind of a word search game. On the paper, there are a lot of different words to find. That's what's going on here. Inside the layer that we got from the file, we need to extract the features that are in it. Each feature is unique and has it's own properties. 

In [19]:
# the layer contains multiple features. we need to iterate over all of them to completely extract all the data
for feature in layer:

    # this iterates through all the properties of that feature and stores it into a dictionary using field_list
    props = {}
    for i in range(len(field_list)):
        props[field_list[i]] = str(feature.GetField(field_list[i]))

    # each feature corresponds to an object. this gives us that object reference and then transforms it to the 
    # coordinate system
    pt = feature.GetGeometryRef()
    pt.Transform(transform)
    
    # gives us our boundaries 
    (temp_x_min, temp_x_max, temp_y_min, temp_y_max) = pt.GetEnvelope()
    if temp_x_min < x_min: x_min = temp_x_min
    if temp_x_max > x_max: x_max = temp_x_max
    if temp_y_min < y_min: y_min = temp_y_min
    if temp_y_max > y_max: y_max = temp_y_max
    
    # actually what we will eventually use to draw the shape on the map. this adds the shapefile information
    # to a geojson file
    polygon = shapely.wkt.loads(pt.ExportToWkt())
    geoJSON = geojson.Feature(geometry=polygon, properties=props)
    
    shapes.append(geoJSON)
    
# compiles the features extracted above into a collection of features ready for dumping the information
# into a geojson file
feature_collection = geojson.FeatureCollection(shapes)

Now to create the map

In [20]:
# creates the map
m = Map(center = ((y_min+y_max)/2, (x_min+x_max)/2), zoom = 4, scroll_wheel_zoom = True)

# creates the widget that displays the shapefile information
box = ipyw.Textarea(layout=ipyw.Layout(width='100%', height='100px'))

# on click listener
def shape_click(**kwargs):
    message = ''
    for key in kwargs['properties']:
        if not type(kwargs['properties'][key]) == 'dict':
            message += key + ' = ' + str(kwargs['properties'][key]) + '\n'
    box.value = message

# adds the shapefile geometry to the ipyleaflet map. styles it such that 
# on mouse hover, the geometry displays blue, otherwise it remains red
geo_json = GeoJSON(data = feature_collection,
                    hover_style = {'fillColor': '#3333ff', 'color': '#6666ff', 'weight': 1.5},
                    style = {'fillColor': '#ff6666', 'color': '#ff3333', 'weight': 1.5})

# adds click listener to geojson shapes
geo_json.on_click(shape_click)

# adds geojson to the map
m.add_layer(geo_json)

In [21]:
display(box)
display(m)

Textarea(value='', layout=Layout(height='100px', width='100%'))

Map(center=[-3.1774349999999956, 2.842170943040401e-14], controls=(ZoomControl(options=['position', 'zoom_in_t…

In [22]:
# for uploading your own shapefile onto jupyter notebook, use this

# import shutil
# from hublib.ui import FileUpload

# # checks to see if there is an existing file. If there is, removes it so that things are simpler
# # this could be changed so that every time a file is uploaded, it gets placed into a different directory

# if os.path.exists(os.getcwd() + '/tmpdir'):
#     for file in os.listdir(os.getcwd() + '/tmpdir'):
#         temp_file_path = os.path.join(os.getcwd() + '/tmpdir', file)
#         if os.path.isfile(temp_file_path):
#             os.remove(temp_file_path)
#         if os.path.isdir(temp_file_path):
#             shutil.rmtree(temp_file_path)

    
# # done is a callback function that will display which files were uploaded upon completion

# def done(w, files):
#     display('{} was uploaded'.format(files))

# # uploads the file    

# f = FileUpload("Select zipped shapefile",
#                "Upload File",
#                cb = done,
#                maxsize = '500M'
#               )
# f

# # gets the file path that we will use to extract files and check to see if they are valid or not

# zip_path = os.path.abspath(f.list()[0])
# folder_path = os.path.dirname(os.path.abspath(f.list()[0]))

ModuleNotFoundError: No module named 'hublib'